# English Parser

"BuT dAvId, ThAt's JuSt A mOnOiDaL-eNdOfUnCtOr-DeRiVeD cOnTeXt-CoErCiNg LaMbDa CoMbInAtOr In SiMpLeR tErMs"

In [23]:
# dependencies

import enum

from xnlp.parser import (
    Parser,
    runParser,
    satisfy,
    optional,
    zero_or_more,
    one_or_more,
    expect_end
)

In this assignment, you are going to explore how to write parsing rules as described in class. The grammar you are going to design is a simplified version of English; words in such language consist of only 6 types:

- `ADJ`: Adjectives (e.g. `beautiful`, `big`)
- `ADV`: Adverbs (e.g. `gracefully`, `quickly`)
- `NOUN`: Noun (e.g. `penguin`, `penguin`, `penguin`)
- `PUNCT`: Punctuation (e.g. `,`, `;`, `.`)
- `VERB`: Verb (e.g. `run`, `eat`, `slap`)
- `CCONJ`: Coordinating Conjunctions (e.g. `but`, `and`, `yet`)

For a more in-depth explanation on the above part-of-speech tags, visit [Universal POS Tags](https://universaldependencies.org/u/pos/all.html).

In [24]:
class T(enum.Enum):
    """
    Some simple tokens. Very inconclusive, but simple.
    """

    ADJ = 'Adjective'
    ADV = 'Adverb'
    NOUN = 'Noun'
    PUNCT = 'Punctuation'
    VERB = 'Verb'
    CCONJ = 'Coordinating Conjunction'

To parse each sentence, we will be using monadic parser combinators to make life easier. Here is some setup code. Feel free to skip if you are not interested.

Note that the parse result will be linearized into a list for simplicity's sake. This assignment focuses more on seeing if your parser can identify grammatically correct sentences from those with grammatical errors than building the parse tree.

In [25]:
wrapped_satisfy = lambda p: satisfy(p).fmap(lambda t: t[0])
word = lambda w: wrapped_satisfy(lambda x: x[0] == w)
tok = lambda t: wrapped_satisfy(lambda x: x[1] == t)
end = expect_end('Expecting end of input')

The tokens in a sentence is already tokenized for you. Tokens are structured as tuples:

```python
('word', T.NOUN)
```

__Tip__: for the most part, you only need to check for the token's type when constructing the parser; the only place where the string content is necessary is when an `Token.PUNCT` is encountered.

Test sentences are the following:

In [26]:
SIMPLE = [
    ('Elephant', T.NOUN),
    ('eat', T.VERB),
    ('banana', T.NOUN),
    ('.', T.PUNCT)
]

SENTS = [
    SIMPLE
]

Building the parser is rather simple; you just need to organize everything into Backus-Naur Form like we discussed during class. To illustrate, here is how you parse a simple noun-verb-noun statement (`+` is the concatenating separator):

In [33]:
NOUN_V_NOUN = tok(T.NOUN) + tok(T.VERB) + tok(T.NOUN) + word('.') + end

Now let's try parsing the simple "Elephant eat banana." sentence!

In [36]:
print('Result:', runParser(NOUN_V_NOUN, SIMPLE))

Result: ['Elephant', 'eat', 'banana', '.']


It's that easy, yay!

You might realize the similarity between our combinators and the BNF syntax; indeed, they captures the same concept. Here are some other tools:

- `optional`: optionally matches a parser (e.g. `optional(tok(T.NOUN))` matches a `Noun` and returns `[<NOUN>]` if possible, otherwise returns `[]`)
- `zero_or_more`: matches a parser as much times as possible, including empty match (e.g. `zero_or_more(tok(T.NOUN))` matches numerous `Noun` and return `[<NOUN> * n]`, where `n` can be `0`)
- `one_or_more`: matches a parser as much times as possible, but at least once
- `end`: matches the end of an input
- `+`: concatenates two parsers; they must match the input string in the given order
- `|`: joins two parsers; the new parser will success if either of the two 